In [2]:
# Libraries

from selenium import webdriver
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time
import uuid
import re

In [3]:
# define classes of information to be collected

class Reviews:
    def __init__(self, review_id, item_id, review_time, review_variant, star_rating, review_content):
        self.review_id = review_id
        self.item_id=item_id
        self.review_time = review_time
        self.review_variant = review_variant
        self.star_rating  = star_rating
        self.review_content = review_content

class Items:
    def __init__(self, item_id, item_name, item_ratings, ratings_num, item_sold, shop_name, item_price, item_url, item_desc, image_url):
        self.item_id=item_id
        self.item_name=item_name
        self.item_ratings=item_ratings
        self.ratings_num=ratings_num
        self.item_sold=item_sold
        self.shop_name=shop_name
        self.item_price=item_price
        self.item_url=item_url
        self.item_desc=item_desc
        self.image_url=image_url


In [4]:
#Create Chrome options
chrome_options= Options()
chrome_options.add_argument('--no sandbox')
chrome_options.add_argument('disable-notifications')
chrome_options.add_argument('disable-inforbars')

# Set the path to the Chrome driver executable
chrome_driver_path = 'D:\Chrome web driver\chromedriver-win64\chromedriver.exe'

# Create an instance of the Chrome browser
browser = webdriver.Chrome(executable_path=chrome_driver_path, options=chrome_options)

# Navigate to a URL 
browser.get("https://shopee.com.my/")

C:\Users\Asus\AppData\Local\Temp\ipykernel_5436\1499064468.py:11: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(executable_path=chrome_driver_path, options=chrome_options)


In [52]:
soup=BeautifulSoup(browser.page_source, "html.parser")

the_links=[]

parent_div = soup.findAll('div', class_="col-xs-2-4 shopee-search-item-result__item")

for parent_div in parent_div:
    a=parent_div.find('a')
    if (a!=None):
        href=a.get('href')
        the_links.append(href)

        # print the links
        print(href)
        

/READY-STOCK💝WEBEE-Hipster-Men-Women's-Kasut-Sandal-Perempuan-Selipar-Wanita-Casual-Rumah-House-Slipper-i.7940214.15038110494?sp_atk=530ae439-1cf9-4b9c-a3c4-d9f2a37d1dde&xptdk=530ae439-1cf9-4b9c-a3c4-d9f2a37d1dde
/L-F-Women-thick-Heels-sandals-perempuan-kasut-(plus-size-35-42)-i.236629981.9433628387?sp_atk=36f8d147-54a0-4341-b5aa-51a2245ca88a&xptdk=36f8d147-54a0-4341-b5aa-51a2245ca88a
/Crocss-Jibbitz-Cartoon-Cute-Animals-2.5D-DIY-Shoes-Charm-Button-洞洞鞋配件鞋飾鞋花智必星-i.468825515.15791603768?sp_atk=725d956b-2067-4a5f-9112-1dbdfcc3a6eb&xptdk=725d956b-2067-4a5f-9112-1dbdfcc3a6eb
/Women's-Sneakers-Shoes-Sport-Shoes-Flat-Kasut-ESWS022-18-Sukan-Wanita-Sekolah-Girl-Perempuan-TRAVEL-i.335708871.7963255926?sp_atk=b04ee792-2346-475d-8058-e86f376f5919&xptdk=b04ee792-2346-475d-8058-e86f376f5919
/READY-STOCKWEBEE-HD-6008-Pinco-Women's-Flat-Shoes-Kasut-Wanita-jelly-sandals-non-slip-beach-hole-i.7940214.7141328216?sp_atk=94bb8b1c-31ed-4130-ac2f-14e2bb81f928&xptdk=94bb8b1c-31ed-4130-ac2f-14e2bb81f928
/MYKUT

In [53]:
len(the_links)

60

In [54]:
items_list=[]
reviews_list=[]

In [55]:
for i in range (9,len(the_links)):
    
    # open link
    links='https://shopee.com.my'+ the_links[i]
    browser.get(links)
    time.sleep(10)

    # parse html using beautifulsoup
    soup=BeautifulSoup(browser.page_source, "html.parser")

    # extract product information
    id=uuid.uuid4()
    title = soup.find('div', attrs={'class':'_44qnta'}).find('span').get_text(strip=True)
    ratings=soup.find('div', class_="_1k47d8 _046PXf").get_text(strip=True)
    ratings_num=soup.select('div._1k47d8:not(div._1k47d8._046PXf)')[0].get_text(strip=True)
    sold=soup.find('div', class_="e9sAa2").get_text(strip=True)
    shop_name=soup.find('div',class_="VlDReK").get_text(strip=True)
    price = soup.find('div', class_='pqTWkA').get_text(strip=True)

    # extract description
    descriptions = soup.find_all('p', class_='irIKAp')
    

    desc_array=[]
    for description in descriptions:
        desc = re.sub(r'<[^>]+>', '', str(description))  # Remove HTML tags
        desc = desc.strip()  # Remove leading/trailing whitespaces
        desc_array.append(desc)

    text = "\n".join(desc_array)

    # extract picture url
    img_tag= soup.find('img', class_='_7D4JtJ')

    if img_tag:
        image = img_tag['src']
    else:
        image=None

    # append item to a list
    items_list.append(Items(id,title,ratings,ratings_num,sold,shop_name, price, links, description, image))

    # reviews extraction
    elements = browser.find_elements(By.CLASS_NAME,"product-rating-overview__filter")
    counter=5
    for j in range(1,6):
        # select star categories
        elements[j].click()
        
        # loop is for each category
        for k in range (0,31): 
            soup=BeautifulSoup(browser.page_source, "html.parser")
            review_containers=soup.findAll('div',class_="shopee-product-rating__main")

            no_reviews_found = False

            for rc in review_containers:
                review_id=uuid.uuid4()
                item_id=id
                time_var=rc.find('div',class_="shopee-product-rating__time").get_text(strip=True).split("|")

                if(len(time_var)==2):
                    review_time=time_var[0]
                    variant=time_var[1]
                else:
                    review_time=time_var[0]
                    variant="None"

                star_rating=counter
                reviews=rc.find('div',class_="Rk6V+3")

                if(reviews!=None):
                    reviews=reviews.get_text(strip=True)
                else:
                    reviews="None"
                    no_reviews_found = True

                reviews_list.append(Reviews(review_id, item_id, review_time,variant,star_rating,reviews))
            
            # no more reviews break
            if no_reviews_found:
                print("No reviews found, breaking the loop")
                break

            time.sleep(1)
            try:
                WebDriverWait(browser, 1).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[class^='shopee-icon-button shopee-icon-button--right "))).click()
            except:
                print("not found")
                break
            time.sleep(1)
        
        counter=counter-1
        time.sleep(2)
    print(i) 
    
    

No reviews found, breaking the loop
No reviews found, breaking the loop
No reviews found, breaking the loop
No reviews found, breaking the loop
9
not found
No reviews found, breaking the loop
No reviews found, breaking the loop
No reviews found, breaking the loop


In [46]:
items_df = pd.DataFrame([vars(obj) for obj in items_list])
reviews_df = pd.DataFrame([vars(obj) for obj in reviews_list])

In [47]:
items_df.shape

(61, 10)

In [48]:
items_df.head()

,item_id,item_name,item_ratings,ratings_num,item_sold,shop_name,item_price,item_url,item_desc,image_url
0,12cdd18a-5ea8-4802-b260-11620b43b049,K3 READY STOCK MYFOOYIN Sandal Kasut Slipper F...,4.9,3.4k,9.5k,Myfooyin,RM8.50 - RM8.80,https://shopee.com.my/K3-READY-STOCK-MYFOOYIN-...,[READY STOCK \nMALAYSIA \nMEASUREMENT:\n37 = 2...,https://down-my.img.susercontent.com/file/9031...
1,d331247e-b55f-4df9-842a-534c79f36c9b,MYKUTSU Rosalie Women Low Heels (Plus Size 35-45),4.9,2.9k,7k,MYKUTSU Shoes Wholesale,RM28.90,https://shopee.com.my/MYKUTSU-Rosalie-Women-Lo...,[#kasut #kasutwanita #kasutperempuan #kasuttin...,https://down-my.img.susercontent.com/file/4847...
2,6fead421-85ad-4823-ad9a-5f9799d05420,Hot Sale 6 Colors Korean Fashion Woman Sport S...,4.9,6.4k,12.7k,Xiaoyulu Shoes Wholesale,RM25.50 - RM28.00,https://shopee.com.my/Hot-Sale-6-Colors-Korean...,[#womanshoes #Sneakers #runningshoes #kasutss...,https://down-my.img.susercontent.com/file/6034...
3,8e5a2cf4-4356-4f16-b41e-99c04f912451,[ZEN] Jibbitz Crocs / Crocs Button / Crocs Pen...,5.0,4.1k,44.4k,Zen Shoes Online Store,RM1.00,https://shopee.com.my/-ZEN-Jibbitz-Crocs-Crocs...,[LOCAL SELLER\nREADY STOCK in Johor. \n\nPenda...,https://down-my.img.susercontent.com/file/2fa4...
4,525ef0aa-d904-4c5d-a7a0-61adc62e021d,K23 READY STOCK MYFOOYIN Kasut Wanita Men’s Sh...,4.9,3.1k,9.1k,Myfooyin,RM6.80,https://shopee.com.my/K23-READY-STOCK-MYFOOYIN...,[READY STOCK \nMALAYSIA\nsize:\n36-37\n38-39\n...,https://down-my.img.susercontent.com/file/b4fc...


In [49]:
reviews_df.shape

(6076, 6)

In [50]:
reviews_df.tail()

,review_id,item_id,review_time,review_variant,star_rating,review_content
6071,37570d5c-7663-4ccd-9399-96eb387ad051,23cedffe-2564-435f-916e-24bc2090799b,2023-05-03 19:40,"Variation: WHITE,36",1,Dirty
6072,fbf3bf9e-2f13-4fe5-a85a-3d9ae31a9a03,23cedffe-2564-435f-916e-24bc2090799b,2023-08-23 12:01,"Variation: PINK,36",1,None
6073,3736b736-270d-4c0d-989e-9fa75adee8d3,23cedffe-2564-435f-916e-24bc2090799b,2023-09-08 11:51,"Variation: BLACK,39",1,None
6074,c897d324-f617-46b1-9d2e-f3e2ab072085,23cedffe-2564-435f-916e-24bc2090799b,2023-07-31 10:01,"Variation: BLUE,39",1,None
6075,6b2d4898-a598-4a56-b95e-cd0bfb9a4ed0,23cedffe-2564-435f-916e-24bc2090799b,2023-06-22 17:22,"Variation: BLUE,39",1,None


In [51]:
reviews_df.to_json('womenshoes_reviews_4.json', orient='records', lines=True, force_ascii=False, default_handler=str)
items_df.to_json('womenshoes_items_4.json', orient='records', lines=True, force_ascii=False, default_handler=str)